In [1]:
import Formatting
include("jlFiles/printmat.jl")

include("jlFiles/lagnPs.jl")
include("jlFiles/excise.jl")
include("jlFiles/OlsFn.jl")

OlsFn (generic function with 1 method)

In [2]:
#xx   = readdlm("Data/FFmFactorsPs.csv",',',header=true)
#x    = xx[1]
#Rme  = x[:,2]
#RSMB = x[:,3]                #small minus big firms
#RHML = x[:,4]                #high minus low book-to-market ratio
#Rf   = x[:,5]                    #interest rate
#
#
#x = readdlm("Data/FF25Ps.csv",',')  #no header line: x is matrix
#R  = x[:,2:end]                    #returns for 25 FF portfolios
#Re = R - repmat(Rf,1,size(R,2))   #excess returns for the 25 FF portfolios
#y = Re(:,[1,25])                 #use only portfolio 1 (small growth) and 25 (large value)
#x = [ones(size(Re,1),1) Rme RSMB RHML]
#------------------------------------------------------------------------------

xx  = readdlm("Data/BondPremiaPs.csv",',',header=true)
z   = xx[1]
rx  = z[:,2:5]
f   = z[:,6:end]

x = [ones(size(f,1),1) lagnPs(f,12)]
K = size(x,2)

yx = excise([rx[:,4] x])
y  = yx[:,1]
x  = yx[:,2:end]

580×6 Array{Float64,2}:
 1.0  3.5166  4.0072  4.1648  4.1854  4.1875
 1.0  3.6469  4.1593  4.1677  4.1677  4.1674
 1.0  3.7582  4.2396  4.2413  4.2409  4.2415
 1.0  3.6066  4.1098  4.2476  4.27    4.2725
 1.0  3.6202  4.0626  4.1609  4.1827  4.1876
 1.0  3.5495  4.0011  4.1152  4.1438  4.1514
 1.0  3.506   3.835   4.0563  4.2055  4.3052
 1.0  3.621   3.9054  4.0863  4.2013  4.2745
 1.0  3.6645  3.9815  4.1077  4.1579  4.1779
 1.0  3.6923  3.9703  4.0941  4.1493  4.174 
 1.0  3.9341  4.0843  4.1391  4.1449  4.1451
 1.0  3.7492  4.0388  4.1766  4.1914  4.1925
 1.0  3.8099  3.9241  4.0311  4.1305  4.2229
 ⋮                                    ⋮     
 1.0  0.2054  0.6784  1.566   2.523   3.3852
 1.0  0.2215  0.7405  1.6477  2.6287  3.5221
 1.0  0.2606  0.415   1.1133  2.0059  2.8972
 1.0  0.1314  0.2478  0.7806  1.4826  2.2086
 1.0  0.1739  0.3967  0.8496  1.3946  1.9447
 1.0  0.1628  0.328   0.8625  1.5331  2.2061
 1.0  0.1573  0.3379  0.8458  1.4754  2.1066
 1.0  0.1493  0.3357  0.7645  1

In [3]:
(bLS,res,yhat,Covb,) = OlsFn(y,x)              #OLS estimate and classical std errors
StdbLS = sqrt(diag(Covb))

println("\nLS coeffs and std")
printmat(round([bLS';StdbLS'],3))

(T,n) = size(y,1,2)                 #no. obs and no. test assets
K     = size(x,2)


LS coeffs and std
    -3.306    -4.209    10.627   -14.397     7.096     1.284
     0.823     0.712     4.509    12.885    15.862     6.898



6

In [4]:
BlockSize = 10                  #size of blocks
NSim      = 2000                 #no. of simulations
srand(123)

nBlocks = round(Int,ceil(T/BlockSize))             #number of blocks, rounded up
bBoot   = fill(NaN,(NSim,K*n))                       #vec(b), [beq1 beq2..beqn]
for i = 1:NSim                                       #loop over simulations
  t_i       = rand(1:T,nBlocks,1)                    #nBlocks x 1, random starting row of blocks
  t_i       = t_i .+ collect(0:BlockSize-1)'         #nBlocks x BlockSize, each row is a block
  vv_i       = t_i .> T
  t_i[vv_i]  = t_i[vv_i] - T                         #wrap around if index > T
  #println(t_i)                                      #uncomment to see which rows that are picked out
  t_i        = vec(t_i')                             #column vector of the blocks
  epsilon    = res[t_i,:]
  epsilon    = epsilon[1:T,:]                        #get exact sample length
  y_i        = x*bLS + epsilon
  b_i,       = OlsFn(y_i,x)                          #,skips the remaining outputs
  bBoot[i,:] = b_i'
end

println("\nAverage bootstrap estimates and bootstrapped std")
printmat([mean(bBoot,1); std(bBoot,1)])


Average bootstrap estimates and bootstrapped std
    -3.318    -4.164    10.636   -14.643     7.371     1.206
     2.072     1.391     8.024    23.175    29.076    12.906

